# Law

> Fill in a module description here

In [ ]:
# | default_exp core.law

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# | export
from __future__ import annotations
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import (
    List,
    Dict,
    Optional,
    Any
)

from vipers.imports import *
from vipers.core.property import Property
from vipers.utils import camel_to_snake

### Law

In [ ]:
# | export
class _BaseLaw(ABC):
    def __init__(self):
        self._properties: Optional[Dict[str, Property]] = None
    
    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @classmethod
    @property
    def snake_name(cls) -> str:  # return the snake style name
        return camel_to_snake(cls.__name__)
    
    @property
    def properties(self) -> Optional[Dict[str, Property]]:
        return self._properties

    def n_known_variables(self, timestep: int) -> int:  # the number of known variables
        n_knowns = 0
        for p in self.properties:
            name = camel_to_snake(p["object"].__name__)
            if timestep in self.compound.properties[name]._data:
                n_knowns += 1
        return n_knowns

    def is_solveable(self, timestep: int) -> bool:  # the timestep
        n_unknowns = len(self.properties)
        n_knowns = self.n_known_variables(timestep)
        print(f"n_unknowns={n_unknowns} and n_knowns={n_knowns}")
        return n_unknowns - n_knowns <= 1

    def _run_config(self) -> None:  # run all configuration
        self._config_properties()

    def _config_properties(self) -> None:  # add law's properties to compound

        for p in self.properties:
            # name = camel_to_snake(p['object'].__name__)
            name = p["object"].snake_name

            if not name in self.compound.properties:
                self.compound.properties[name] = p["object"](compound=self.compound)

            self.compound.properties[name].add_law(self)

    def solve(
        self,
        timestep: int,  # the time step of the unknown variable
        unknown: str,  # the unknown variable that you want to solve
    ) -> smp.solve:
        unknown_symbol = self.compound.properties[unknown].symbol
        return smp.solve(self.expression, unknown_symbol)

    def __call__(self, *args, **kwargs):
        return self.expr(*args, **kwargs)

    # @abstractmethod
    # def expr(self):
    #     """The symbolic expression of the law"""
    #     pass

    def __repr__(self):
        return f"Law({self.name})"

In [ ]:
b = _BaseLaw()

In [ ]:
b._properties

In [ ]:
# | export
class Law(_BaseLaw):
    pass